<a href="https://colab.research.google.com/github/AristiPap/Thesis_Stuff/blob/main/BoundaryAttack_Aria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from __future__ import print_function
try:
	raw_input
except:
	raw_input = input

from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

import matplotlib.pyplot as plt
import numpy as np



import numpy as np
import time
import os
from PIL import Image
import sys
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

RESNET_MEAN = np.array([103.939, 116.779, 123.68])

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



![](https://drive.google.com/uc?export=view&id=1YMexys51Bbmkb-JDdFF1AiKWOpot-SEE)


![](https://drive.google.com/uc?export=view&1YMexys51Bbmkb-JDdFF1AiKWOpot-SEE)


In [15]:
def sphere_perturbation(delta, prev_sample, target_sample):
    """Generate orthogonal perturbation."""
    perturb = np.random.randn(1, 224, 224, 3)
    perturb /= np.linalg.norm(perturb, axis=(1, 2))
    perturb *= delta * np.mean(get_diff(target_sample, prev_sample))
    # Project perturbation onto sphere around target
    # Orthorgonal vector to sphere surface
    diff = (target_sample - prev_sample).astype(np.float32)
    diff /= get_diff(target_sample, prev_sample)  # Orthogonal unit vector
    # We project onto the orthogonal then subtract from perturb
    # to get projection onto sphere surface
    perturb -= (np.vdot(perturb, diff) / np.linalg.norm(diff)**2) * diff
    # Check overflow and underflow
    overflow = (prev_sample + perturb) - 255 + RESNET_MEAN
    perturb -= overflow * (overflow > 0)
    underflow = -RESNET_MEAN
    perturb += underflow * (underflow > 0)
    return perturb

In [16]:
def forward_perturbation(epsilon, prev_sample, target_sample):
    """Generate forward perturbation."""
    perturb = (target_sample - prev_sample).astype(np.float32)
    perturb /= get_diff(target_sample, prev_sample)
    perturb *= epsilon
    return perturb


def get_converted_prediction(sample, classifier):
    """
    The loss of precision often causes the label of the image to change, particularly
    because we are very close to the boundary of the two classes.
    This function checks for the label of the exported sample
    by simulating the export process.
    """
    sample = (sample + RESNET_MEAN).astype(np.uint8).astype(np.float32) - RESNET_MEAN
    label = decode_predictions(classifier.predict(sample), top=1)[0][0][1]
    return label


def save_image(sample, classifier, folder):
    """Export image file."""
    label = get_converted_prediction(np.copy(sample), classifier)
    sample = sample[0]
    sample += RESNET_MEAN
    sample = sample[..., ::-1].astype(np.uint8)
    # Convert array to image and save
    sample = Image.fromarray(sample)
    id_no = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    # Save with predicted label for image (may not be adversarial due to uint8 conversion)
    sample.save(os.path.join("/content/drive/MyDrive/my_boundary_attack/boundary-attack-master/images", folder,"{}_{}.png".format(id_no, label)))

#preprocessing with keras
def preprocess(sample_path):
    """Load and preprocess image file."""
    img = image.load_img(sample_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

#keep mean error for each iteration
def get_diff(sample_1, sample_2):
    """Channel-wise norm of difference between samples."""
    sample_1 = sample_1.reshape(3, 224, 224)
    sample_2 = sample_2.reshape(3, 224, 224)
    diff = []
    for i, channel in enumerate(sample_1):
      diff.append(np.linalg.norm((channel - sample_2[i]).astype(np.float32)))
    return np.array(diff)
    #return np.linalg.norm(sample_1 - sample_2, axis=(1, 2))

In [23]:
def boundary_attack():
    # Load model, images and other parameters
    classifier = ResNet50(weights='imagenet')
    initial_sample = preprocess('/content/drive/MyDrive/my_boundary_attack/boundary-attack-master/images/original/awkward_moment_seal.png')
    target_sample = preprocess('/content/drive/MyDrive/my_boundary_attack/boundary-attack-master/images/original/bad_joke_eel.png')
    folder = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    os.mkdir(os.path.join("/content/drive/MyDrive/my_boundary_attack/boundary-attack-master/images", folder))
    save_image(np.copy(initial_sample), classifier, folder)
    #classifier initialised with our picture (in this case picture of seal that you can faintly see behind the eel when the attack is executed)
    attack_class = np.argmax(classifier.predict(initial_sample))
    #Our target image is the eel.The who,e point is to create an image that looks almost exactly like the eel but has pertrubations "inspired by the eel", 
    #so that it's misclassified
    target_class = np.argmax(classifier.predict(target_sample))

    adversarial_sample = initial_sample
    n_steps = 0
    n_calls = 0
    epsilon = 1.
    delta = 0.1

    # Move first step to the boundary
    # get first adversarial point to begin initiating the attack 
    while True:
        trial_sample = adversarial_sample + forward_perturbation(epsilon * get_diff(adversarial_sample, target_sample), adversarial_sample, target_sample)#adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample)
        prediction = classifier.predict(trial_sample.reshape(1, 224, 224, 3))#classifier.predict(trial_sample)
        
        n_calls += 1
        if np.argmax(prediction) == attack_class:
          adversarial_sample = trial_sample
          break
        else:
          epsilon *= 0.9

    # Iteratively run attack
    while True:
        print("Step #{}...".format(n_steps))
        # Orthogonal step
        print("\tDelta step...")
        d_step = 0
        while True:
            d_step += 1
            print("\t#{}".format(d_step))
            trial_samples = []
            for i in np.arange(10):
                trial_sample = adversarial_sample + sphere_perturbation(delta, adversarial_sample, target_sample)
                trial_samples.append(trial_sample)
            
            trial_samples = np.asarray(trial_samples).reshape((-1,224,224,3))
            predictions = classifier.predict(trial_samples)
            n_calls += 10
            predictions = np.argmax(predictions, axis=1)
            d_score = np.mean(predictions == attack_class)
            if d_score > 0.0:
                if d_score < 0.3:
                    delta *= 0.9
                elif d_score > 0.7:
                    delta /= 0.9
                adversarial_sample = np.array(trial_samples)[np.where(predictions == attack_class)[0][0]]
                break
            else:
                delta *= 0.9
        # Forward step
        print("\tEpsilon step...")
        e_step = 0
        while True:
            e_step += 1
            print("\t#{}".format(e_step))
            trial_sample = adversarial_sample + forward_perturbation(epsilon, adversarial_sample, target_sample)
            prediction = classifier.predict(trial_sample)
            n_calls += 1
            #adversarial step
            if np.argmax(prediction) == attack_class:
                adversarial_sample = trial_sample
                epsilon /= 0.5
                break
            elif e_step > 500:
                break
            else:
                epsilon *= 0.5

        n_steps += 1
        # keep some checkpoints so that we save the pictures whilst the attack progresses
        chkpts = [250, 750]
        if n_steps in chkpts:
            print("{} steps".format(n_steps))
            save_image(np.copy(adversarial_sample), classifier, folder)
        diff = np.mean(get_diff(adversarial_sample, target_sample))
        #stop at 1000 steps ~= 40 minutes
        if diff <= 1e-3 or e_step > 500 or n_steps > 1000:
            print("{} steps".format(n_steps))
            print("Mean Squared Error: {}".format(diff))
            save_image(np.copy(adversarial_sample), classifier, folder)
            break

        print("Mean Squared Error: {}".format(diff))
        print("Calls: {}".format(n_calls))
        print("Attack Class: {}".format(attack_class))
        print("Target Class: {}".format(target_class))
        print("Adversarial Class: {}".format(np.argmax(prediction)))

In [24]:
if __name__ == "__main__":
    boundary_attack()

Step #0...
	Delta step...
	#1
	Epsilon step...
	#1
Mean Squared Error: 6706.3359375
Calls: 16
Attack Class: 150
Target Class: 390
Adversarial Class: 150
Step #1...
	Delta step...
	#1
	#2


KeyboardInterrupt: ignored